* (6 points) Create your class that implements the Gradient Boosting concept, based on the locally weighted regression method (Lowess class), and that allows a user-prescribed number of boosting steps. The class you develop should have all the mainstream useful options, including “fit,” “is_fitted”,  and “predict,” methods.  Show applications with real data for regression, 10-fold cross-validations and compare the effect of different scalers, such as the “StandardScaler”, “MinMaxScaler”, and the “QuantileScaler”.  In the case of the “Concrete” data set, determine a choice of hyperparameters that yield lower MSEs for your method when compared to the eXtream Gradient Boosting library.

* (3 points) Based on the Usearch library, create your own class that computes the k_Nearest Neighbors for Regression.

* (1 point) Host your project on your GitHiub page.

In [1]:
# computational libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.interpolate import interp1d, RegularGridInterpolator, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [2]:
s_scale = StandardScaler()
q_scale = QuantileTransformer()
mm_scale = MinMaxScaler()

In [3]:
from google.colab import files
files = files.upload()

Saving concrete.csv to concrete.csv


In [4]:
data = pd.read_csv('concrete.csv')
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

In [28]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

## Q1

In [6]:
# Gaussian Kernel
def Gaussian(w):
  return np.where(w>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*w**2))

# Tricubic Kernel
def Tricubic(w):
  return np.where(w>1,0,70/81*(1-w**3)**3)

# Quartic Kernel
def Quartic(w):
  return np.where(w>1,0,15/16*(1-w**2)**2)

# Epanechnikov Kernel
def Epanechnikov(w):
  return np.where(w>1,0,3/4*(1-w**2))

In [7]:
def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

In [23]:
from scipy.spatial.distance import cdist


In [22]:
def weight_function(u,v,kern=Gaussian,tau=0.5):
    return kern(cdist(u, v, metric='euclidean')/(2*tau))

In [20]:
import xgboost
from sklearn import linear_model

In [21]:
def lw_ag_md(x, y, xnew,f=2/3,iter=3, intercept=True):

  n = len(x)
  r = int(ceil(f * n))
  yest = np.zeros(n)

  if len(y.shape)==1: # here we make column vectors
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]
  # dist(x,x) is always symmetric
  w = np.clip(dist(x,x) / np.array(h), 0.0, 1.0)
  # note that w is a square matrix and in Python arithmetic operations such as
  # w**3 or 1-w**3 are performed element-wise
  #w = (1-w**3)**3 # a Tricubic kernel
  w = Epanechnikov(w)

  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(delta).dot(np.diag(w[i,:]))
      # when we multiply two diagonal matrices we get also a diagonal matrix
      b = np.transpose(x1).dot(W).dot(y)
      A = np.transpose(x1).dot(W).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1]) # if we want L2 regularization for solving the system
      beta = linalg.solve(A, b)

      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta.ravel())

    residuals = y.ravel() - yest
    s = np.median(np.abs(residuals))

    delta = np.clip(residuals / (6.0 * s), -1, 1)

    delta = (1 - delta ** 2) ** 2

  # here we are making predictions for xnew by using an interpolation and the predictions we made for the train data
  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ Pp')
      f = LinearNDInterpolator(tri,yest[ind])
      output[i] = f(pca.transform(xnew[i].reshape(1,-1)))
      # the output may have NaN's where the data points from xnew are outside the convex hull of X

  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,yest.ravel())
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

In [25]:
class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = []
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test.append(lm.predict([x_new[i]]))
        return np.array(yest_test).flatten()

In [11]:
xtrain, xtest, ytrain, ytest = tts(x,y,test_size=0.3,shuffle=True,random_state=123)

In [12]:
#mse ~ 58

In [13]:
scalers = [s_scale,q_scale,mm_scale]

In [31]:
mean_mse_lwr = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_1 = Lowess(kernel= Gaussian,tau=0.14)
model_2 = Lowess(kernel= Epanechnikov,tau=0.05)
for scale in scalers:
  mse_lwr = []
  for idxtrain, idxtest in kf.split(x):
    xtrain = x[idxtrain]
    ytrain = y[idxtrain].ravel()
    ytest = y[idxtest].ravel()
    xtest = x[idxtest]
    xtrain = scale.fit_transform(xtrain)
    xtest = scale.transform(xtest)

    model_1.fit(xtrain,ytrain)
    yhat_train = model_1.predict(xtrain)
    residuals_train = ytrain - yhat_train
    model_2.fit(xtrain,residuals_train)
    residuals_hat = model_2.predict(xtest)
    yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

    mse_lwr.append(mse(ytest,yhat_lw))
  mean_mse_lwr.append(np.mean(mse_lwr))


print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(mean_mse_lwr))


The Cross-validated Mean Squared Error for Locally Weighted Regression is : [167.06646876133357, 22.09903664018056, 36.616417475594915]


In [32]:
import xgboost

In [33]:
model_xgboost = xgboost.XGBRFRegressor(n_estimators=200,max_depth=7)

In [34]:
model_xgboost.fit(xtrain,ytrain)
mse(ytest,model_xgboost.predict(xtest))

27.443587005918328

## Using a combination of epanechnikov and gaussian kernels and the quantile scaler, i was able to get the boosted regressor to an mse of 22, better than the xgboost benchmark of 27

## Q2

In [35]:
pip install usearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.3 MB/s eta 0:00:00


In [36]:
import numpy as np
import usearch.index
from usearch.index import search, MetricKind, Matches, BatchMatches, Index

In [37]:
vectors = np.random.rand(100, 5).astype(np.float32)
vector = np.random.rand(5).astype(np.float32)

In [38]:
one_in_many: Matches = search(vectors, vector, 5, MetricKind.L2sq, exact=True)

In [39]:
type(one_in_many[0])

usearch.index.Match

In [40]:
len(vectors)

100

In [41]:
vectors = np.random.rand(1000, 100).astype(np.float32)
len(vectors)
vectors.size

100000

In [42]:
class usearch_knn:
    def __init__(self ,max_cs = None,min_cs = None):
        self.max_cs = max_cs
        self.min_cs = min_cs

    def nearest_to_one(self, vectors, vector, n = 15):
      result: BatchMatches = search(vectors, vector, 10, MetricKind.L2sq, exact=True)
      return result.to_list()


    def knn_cluster(self,vectors):
      index = Index(
          ndim=vectors.shape[-1],
          metric='cos'
        )
      n = len(vectors)
      keys = [i for i in range(n)]
      index.add(keys,vectors)
      clustering = index.cluster(
          vectors = vectors,

          min_count= self.min_cs,
          max_count= self.max_cs
      )
      return clustering





In [43]:
uknn = usearch_knn()
clustering = uknn.knn_cluster(vectors)
clustering

usearch.Clustering(for 1000 queries)

In [44]:
centroid_keys, sizes = clustering.centroids_popularity

In [45]:
len(centroid_keys)

59

In [46]:
clusters = []
for i in range(len(centroid_keys)):
    indices = clustering.members_of(centroid_keys[i])
    clusters.append(indices)

for i, cluster_indices in enumerate(clusters):
    print(f"Cluster {i}: {cluster_indices}")

Cluster 0: [  6 113 210 284 285 382 390 520 604 699 861 904]
Cluster 1: [  4   8  22  28 276 282 343 432 534 579 590 672 792 807 847]
Cluster 2: [ 16  69  84 121 126 131 148 273 288 316 329 335 352 394 474 487 544 581
 603 691 749 766 786 793 810 813 816 928 931 934]
Cluster 3: [ 15  29 278 317 418 742]
Cluster 4: [ 18  38  40  71 152 160 199 229 242 265 274 280 369 456 503 505 655 689
 711 714 796 834 852 945 953 999]
Cluster 5: [ 53  82 932]
Cluster 6: [109]
Cluster 7: [  7  44  45  48  57  74  80  95 100 119 125 183 191 194 195 205 212 213
 214 220 250 262 263 267 290 305 345 360 361 370 372 377 402 407 409 414
 426 427 429 467 507 513 521 535 541 545 549 551 556 558 607 618 631 634
 646 657 663 674 675 685 693 705 713 719 722 725 738 773 775 782 784 787
 848 864 865 875 881 888 896 898 906 918 924 943 949 970 973 976 978]
Cluster 8: [ 39  72 140 171 252 277 318 391 433 448 452 453 527 548 630 673 676 684
 878 915 950]
Cluster 9: [ 55 151 180 291 342 373 419 926]
Cluster 10: [ 19  2

In [47]:
vectors = np.random.rand(10000, 1024).astype(np.float32)
vector = np.random.rand(1024).astype(np.float32)

In [48]:
uknn.nearest_to_one(vectors,vector)

[(1407, 150.5949249267578),
 (792, 151.77407836914062),
 (2505, 151.8466339111328),
 (4716, 152.25698852539062),
 (4641, 152.3251190185547),
 (5956, 152.78082275390625),
 (6890, 152.8309326171875),
 (3278, 152.92190551757812),
 (4350, 153.02426147460938),
 (7683, 153.09165954589844)]